<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_1_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introducción

Según la mecánica newtoniana, una partícula no puede seguir una trayectoria curva a menos que sobre ella actúe una cierta aceleración como consecuencia de la acción de una fuerza, ya que si esta no existiese, su movimiento sería rectilíneo. Asimismo, una partícula en movimiento rectilíneo solo puede cambiar su velocidad bajo la acción de una aceleración en la misma dirección de su velocidad (dirigida en el mismo sentido si acelera; o en sentido contrario si desacelera). 

En mecánica clásica se define la aceleración como la variación de la velocidad respecto al tiempo (común a todos los observadores)

>$\frac{vf-vi}{tf-ti} = {a}$

#2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 6.0MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621010 sha256=bd061fa1c57dca5814fdf002116dc1c481d4aadeaf97b63b6bcb3647f1b724cd
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=d2e06de046a22de5b9d7976ca979119d7f24a6fddae918230060289ab6d1e06f
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [13]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidadVehiculos =   3#@param {type: "integer", min:0}
tiempoSegundos =   3 #@param {type:"slider", min:0, max:100, step:1}

# --------------------------------------------

if cantidadVehiculos < 0:
  raise ValueError("La cantidad de vehículos debe ser mayor a 0");

if tiempoSegundos == 0:
  raise ValueError("El tiempo debe ser mayor a cero");

from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy

tiempo_total = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#Defino variables
acelTotal_cpu = 0
vMax = 120 # [m/s]

# --------------------------------------------
# CPU - Defino la memoria de los vectores en cpu.

vi_cpu = numpy.random.randint(vMax, size=cantidadVehiculos)
vi_cpu = vi_cpu.astype( numpy.float32() )

vf_cpu = numpy.random.randint(vMax, size=cantidadVehiculos)
vf_cpu = vf_cpu.astype( numpy.float32() )
acel_cpu = numpy.empty_like( vf_cpu )

# CPU - reservo la memoria GPU.
vf_gpu = cuda.mem_alloc( vf_cpu.nbytes )
vi_gpu = cuda.mem_alloc( vi_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( vf_gpu, vf_cpu )
cuda.memcpy_htod( vi_gpu, vi_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_acel( int n, float time, float *vf, float *vi)
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
    vf[idx] = (vf[idx] - vi[idx])/time;
  }
}
""") 

# CPU - Genero la función kernel.
kernel = module.get_function("kernel_acel")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
dim_hilo = 256
dim_bloque = numpy.int( (cantidadVehiculos+dim_hilo-1) / dim_hilo )

kernel(numpy.int32(cantidadVehiculos),numpy.float32(tiempoSegundos), vf_gpu, vi_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( acel_cpu, vf_gpu )

try: 
  acelPromedio = numpy.mean(acel_cpu) 

  tiempo_total = datetime.now() - tiempo_total

  print( "Cantidad de elementos: ", cantidadVehiculos )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )
  print("Aceleración promedio: ", acelPromedio, "[m/s2]" )

except ValueError as valerr:
  print (valerr)
                       

Cantidad de elementos:  3
Thread x:  256 , Bloque x: 1
Tiempo CPU:  2.981 [ms]
Tiempo GPU:  0.236 [ms]
Aceleración promedio:  19.444445 [m/s2]


#4 Tabla de pasos

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  @param                | Lectura del tiempo.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes vf, vi y acel.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria vf a CPU memoria acel.
CPU      | numpy.mean(vf_cpu)     | Realiza el cálculo del promedio, guardando el valor en acelPromedio 
CPU      |  print()               | Informo los resultados.

#5 Conclusiones


#6 Bibliografía

[1] Aceleracion: [Referencia](https://www.fisicalab.com/apartado/aceleracion)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)

[4] Sliders Parametros: [Referencia](https://colab.research.google.com/notebooks/forms.ipynb)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)